In [1]:
import ray
ray.init(ignore_reinit_error=True, _temp_dir='/tmp/ray_rachel')

2024-01-16 11:06:58,545	INFO services.py:1476 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.7.12', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '10.7.30.230', 'raylet_ip_address': '10.7.30.230', 'redis_address': None, 'object_store_address': '/tmp/ray_rachel/session_2024-01-16_11-06-56_315861_389651/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray_rachel/session_2024-01-16_11-06-56_315861_389651/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray_rachel/session_2024-01-16_11-06-56_315861_389651', 'metrics_export_port': 65236, 'gcs_address': '10.7.30.230:59047', 'address': '10.7.30.230:59047', 'node_id': '665d6feede38254ed9936dba6af9bb2d4bd7c8e866f4959a2285ac7b'})

In [2]:
@ray.remote(num_cpus=1)
def _count_single_region_set_for_single_allc(sample, allc_path, mc_types, region_path, obs_dim, region_dim):
    """Get cell-by-region-by-mc_types count matrix, save to zarr."""
    total_mc_types = list(set(mc_types))

    with pysam.TabixFile(allc_path) as allc:
        region_ids = []
        sample_data = []
        region_chunks = pd.read_csv(region_path, index_col=3, sep='\t', chunksize=2000)
        for chunk in region_chunks:
            region_ids += chunk.index.tolist()
            for _, (chrom, start, end) in chunk.iterrows():
                count_quant = _CountQuantifier(mc_types=total_mc_types)
                try:
                    allc_lines = allc.fetch(chrom, start, end)
                    for line in allc_lines:
                        count_quant.read_line(line)
                except ValueError:
                    # got value error, this chrom not exist in allc
                    pass
                sample_data.append(count_quant.summary())
        data = xr.DataArray(
            np.array([sample_data]),
            coords=[[sample], region_ids, total_mc_types, ["mc", "cov"]],
            dims=[obs_dim, region_dim, "mc_type", "count_type"],
        )
        return data

In [3]:
import pandas as pd
import pysam
import glob
import pathlib
import xarray as xr
from ALLCools.count_matrix.dataset import _CountQuantifier
import time
import subprocess
import numpy as np
from tqdm import tqdm
import os
import joblib

In [4]:
mc_types = ['CGN']
allc_paths_dir = f"{os.getcwd()}/Total.Balanced.Merged.Allc"
kind = '2-BothGender.CellType.Age'
obs_dim='sample'
region_dim='dmr_region'

In [5]:
ready_celltypes = [pathlib.Path(path).name.split('.')[0] for path in glob.glob(f'{kind}/DMR/*')]
len(ready_celltypes)

8

In [6]:
pathlib.Path('DMR-with-id').mkdir(parents=True, exist_ok=True)

In [7]:
# for celltype in tqdm(ready_celltypes):
#     dmr_bed = pd.read_csv(f'{kind}/DMR/{celltype}.DMR.bed', sep = '\t', header = None)
#     dmr_bed['id'] = dmr_bed[0] + '-' + dmr_bed[1].astype(str) + '-' + dmr_bed[2].astype(str)
#     dmr_bed.to_csv(f'DMR-with-id/{celltype}.with-ID.DMR.bed', sep = '\t', index= None, header = None)

In [25]:
use_cts = joblib.load('/home/qzeng/project/aging/metadata/mc_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

In [28]:
ready_celltypes

['Astro-NT_NN',
 'Astro-TE_NN',
 'LDT-PCG-CS_Gata3_Lhx1_Gaba',
 'MEA-COA_Glut',
 'PAG-PPN_Pax5_Sox21_Gaba',
 'PB_Evx2_Glut',
 'STR_D1_Sema5a_Gaba',
 'Vip_Gaba']

In [27]:
for celltype in tqdm(ready_celltypes):
    df = pd.DataFrame()
    df['allc_path'] = glob.glob(f'{allc_paths_dir}/{celltype}.*.CGN-Merge.allc.tsv.gz')
    df['allc_path'] = df['allc_path'].apply(lambda x: str(pathlib.Path(x).resolve()) )
    df['kind'] = 'BothGender.CellType.Age'
    #df['sample'] = df.apply(lambda x: x['allc_path'].split('/')[-1].replace(x['kind']+'.','').replace('.CGN-Merge.allc.tsv.gz',''), axis=1)
    df['sample'] = df['allc_path'].apply(lambda x: x.split('/')[-1].split('.Both-Gender.')[0])
    allc_table = df[['sample','allc_path']].copy()
    allc_table = allc_table.set_index('sample')['allc_path'].to_dict()
    
    region_path = f'DMR-with-id/{celltype}.with-ID.DMR.bed'
    
    results = [_count_single_region_set_for_single_allc.remote(sample, allc_path, mc_types, region_path, obs_dim, region_dim)
           for sample, allc_path in allc_table.items()]
    total_data = ray.get(results)

    total_data = xr.Dataset({f"{region_dim}_da": xr.concat(total_data, dim=obs_dim)}) 
    
    output_path = f'mC_RAW_DMR_ZARR/{celltype}'
    output_path = pathlib.Path(output_path)
    
    total_data.to_zarr(output_path, mode='w')

100%|██████████████████████████████████████████████████████████████████████████████| 8/8 [06:53<00:00, 51.65s/it]
